<h1> Import </h1>

In [1]:
from pandas import read_csv
from pandas import DataFrame
from math import pow,sqrt,fabs
from sklearn import metrics
import numpy as np
import datetime
import random

<p> <b> Train Data </b> is used to search for best parameter / configuration. </p>
<p> <b> Test Data </b> is actual testing. </p>

<h1> Train Data - Load </h1>

In [2]:
trainData = read_csv("CencusIncome.csv", header = None)
trainLabel = trainData.loc[:,6]
trainLabel, trainLabelLevel = trainLabel.factorize()
trainData = trainData.drop(6, axis = 1)

<h1> Train Data - Preprocessing </h1>

In [3]:
def euclidean_distance(instance1,instance2):
    columnCount = len(instance1)
    sum = 0
    for i in range(columnCount):
        sum += pow(instance1[i]-instance2[i],2)
    return sqrt(sum)

def manhattan_distance(instance1,instance2):
    columnCount = len(instance1)
    sum = 0
    for i in range(columnCount):
        sum += fabs(instance1[i]-instance2[i])
    return sum

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

In [4]:
normalizedTrain = normalize(trainData) 
print(normalizedTrain)

                  0         1             2         3         4             5
0      9.946924e-07  0.001977  3.315641e-07  0.000055  0.000000  1.020197e-06
1      1.275247e-06  0.002125  3.315641e-07  0.000000  0.000000  3.315641e-07
2      9.691875e-07  0.005500  2.295444e-07  0.000000  0.000000  1.020197e-06
3      1.351761e-06  0.005987  1.785345e-07  0.000000  0.000000  1.020197e-06
4      7.141381e-07  0.008631  3.315641e-07  0.000000  0.000000  1.020197e-06
5      9.436825e-07  0.007258  3.570691e-07  0.000000  0.000000  1.020197e-06
6      1.249742e-06  0.004086  1.275247e-07  0.000000  0.000000  4.080789e-07
7      1.326257e-06  0.005347  2.295444e-07  0.000000  0.000000  1.147722e-06
8      7.906529e-07  0.001168  3.570691e-07  0.000359  0.000000  1.275247e-06
9      1.071207e-06  0.004067  3.315641e-07  0.000132  0.000000  1.020197e-06
10     9.436825e-07  0.007153  2.550493e-07  0.000000  0.000000  2.040395e-06
11     7.651480e-07  0.003604  3.315641e-07  0.000000  0.000000 

In [5]:
len(normalizedTrain.index)

32561

<h1> Clustering </h1>

In [6]:
def classify_k_medoids(instance, medoids):
    row_count = len(medoids.index)
    distances = np.zeros(row_count)
    for index, row in medoids.iterrows():
        distances[index] = manhattan_distance(instance,row)
    result = [0, 0]
    result[0] = np.argmin(distances)
    result[1] = distances[np.argmin(distances)]
    return result

def k_medoids(df, k, max_iterations):
    row_count = len(df.index)
    col_count = len(df.columns)
    medoids = df.sample(k)
    medoids = medoids.reset_index(drop=True)
    is_convergence = False;
    i = 0
    error = 0.0
    iteration = 0
    #Initiate array for membership
    membership = []
    for index in range(0,k):
        membership.append([])
    #First time classify
    print(i)
    print(datetime.datetime.now())
    prev_medoids = medoids.copy()
    pred = np.zeros(row_count).astype(int)
    #Classify each row & count error
    for index, row in df.iterrows():
        tmp_array = classify_k_medoids(row, prev_medoids)
        pred[index] = tmp_array[0]
        membership[tmp_array[0]].append(index)
        error += tmp_array[1]
    best_error = error
    best_pred = np.copy(pred)
    best_medoids = medoids.copy()
    print(best_error)
    print(prev_medoids)
    i += 1
    
    while (not is_convergence):
        #Get new medoids (randomize)
        random_class = random.randint(0, k-1)
        sum_members_of_class = len(membership[random_class])
        random_medoid = random.randint(0, sum_members_of_class-1)
        #print("membership " + str(membership[random_class]))
        print("sum member of class" + str(sum_members_of_class))
        print("random medoid" + str(random_medoid))
        for index in range(0, col_count):
            medoids.iat[random_class, index] = df.iat[random_medoid, index]
        print(prev_medoids)
        print(medoids)
        #Initiate array for membership
        membership = []
        for index in range(0,k):
            membership.append([])
        #Classify each row & count error
        print(i)
        print(datetime.datetime.now())
        prev_medoids = medoids.copy()
        pred = np.zeros(row_count).astype(int)
        error = 0.0        
        for index, row in df.iterrows():
            tmp_array = classify_k_medoids(row, prev_medoids)
            pred[index] = tmp_array[0]
            membership[tmp_array[0]].append(index)
            error += tmp_array[1]
        print(error)

        #Stop condition    
        #print(membership)
        if(error >= best_error):
            iteration += 1
        else:
            iteration = 0
            best_error = error
            best_pred = np.copy(pred)
            best_medoids = medoids.copy()
        i += 1
        if(iteration == max_iterations):
            is_convergence = True
    print("best error = " + str(best_error))
    print("best pred = " + str(best_pred))
    print("best medoids = " + str(best_medoids))
    return {"prediction": best_pred, "medoids": best_medoids} 

In [8]:
trainResult = k_medoids(normalizedTrain, 2, 500)

0
2017-12-02 22:57:05.145000
1103.8916793
              0         1             2    3    4             5
0  1.071207e-06  0.004710  2.805543e-07  0.0  0.0  7.651480e-07
1  7.396431e-07  0.003432  3.570691e-07  0.0  0.0  5.100987e-07
sum member of class13139
random medoid12743
              0         1             2    3    4             5
0  1.071207e-06  0.004710  2.805543e-07  0.0  0.0  7.651480e-07
1  7.396431e-07  0.003432  3.570691e-07  0.0  0.0  5.100987e-07
          0         1             2    3    4             5
0  0.000001  0.004710  2.805543e-07  0.0  0.0  7.651480e-07
1  0.000001  0.008385  2.295444e-07  0.0  0.0  1.020197e-06
1
2017-12-02 22:57:16.611000
1125.1393815
sum member of class25838
random medoid17869
          0         1             2    3    4             5
0  0.000001  0.004710  2.805543e-07  0.0  0.0  7.651480e-07
1  0.000001  0.008385  2.295444e-07  0.0  0.0  1.020197e-06
          0         1             2    3    4             5
0  0.000001  0.005998  7

1729.98010087
sum member of class2740
random medoid379
          0         1             2    3    4             5
0  0.000001  0.007516  2.295444e-07  0.0  0.0  1.020197e-06
1  0.000001  0.009928  2.550493e-07  0.0  0.0  8.926727e-07
              0         1             2    3    4         5
0  1.326257e-06  0.007516  2.295444e-07  0.0  0.0  0.000001
1  9.691875e-07  0.004410  2.295444e-07  0.0  0.0  0.000001
19
2017-12-02 23:01:21.349000
1100.50505516
sum member of class24117
random medoid1590
              0         1             2    3    4         5
0  1.326257e-06  0.007516  2.295444e-07  0.0  0.0  0.000001
1  9.691875e-07  0.004410  2.295444e-07  0.0  0.0  0.000001
          0         1             2    3    4             5
0  0.000001  0.007516  2.295444e-07  0.0  0.0  1.020197e-06
1  0.000002  0.005287  2.550493e-07  0.0  0.0  4.080789e-07
20
2017-12-02 23:01:35.178000
1216.86810931
sum member of class25470
random medoid8644
          0         1             2    3    4      

1074.81437319
sum member of class10474
random medoid435
              0         1             2    3    4         5
0  5.866135e-07  0.004645  2.550493e-07  0.0  0.0  0.000001
1  1.377266e-06  0.002383  3.315641e-07  0.0  0.0  0.000001
              0         1             2    3    4         5
0  5.866135e-07  0.004645  2.550493e-07  0.0  0.0  0.000001
1  8.416628e-07  0.005315  3.570691e-07  0.0  0.0  0.000001
38
2017-12-02 23:05:34.595000
1195.32412522
sum member of class19361
random medoid4468
              0         1             2    3    4         5
0  5.866135e-07  0.004645  2.550493e-07  0.0  0.0  0.000001
1  8.416628e-07  0.005315  3.570691e-07  0.0  0.0  0.000001
              0         1             2    3    4         5
0  6.121184e-07  0.007025  2.805543e-07  0.0  0.0  0.000001
1  8.416628e-07  0.005315  3.570691e-07  0.0  0.0  0.000001
39
2017-12-02 23:05:46.062000
1234.05543673
sum member of class24808
random medoid103
              0         1             2    3    4  

1218.5964507
sum member of class15850
random medoid1690
          0         1             2    3    4             5
0  0.000002  0.004692  2.295444e-07  0.0  0.0  2.040395e-07
1  0.000001  0.004292  3.315641e-07  0.0  0.0  1.020197e-06
              0         1             2    3    4             5
0  1.759840e-06  0.004692  2.295444e-07  0.0  0.0  2.040395e-07
1  6.376233e-07  0.000727  2.295444e-07  0.0  0.0  1.020197e-06
56
2017-12-02 23:09:06.435000
1142.28947098
sum member of class26085
random medoid16181
              0         1             2    3    4             5
0  1.759840e-06  0.004692  2.295444e-07  0.0  0.0  2.040395e-07
1  6.376233e-07  0.000727  2.295444e-07  0.0  0.0  1.020197e-06
              0         1             2    3    4         5
0  1.530296e-06  0.002379  2.295444e-07  0.0  0.0  0.000001
1  6.376233e-07  0.000727  2.295444e-07  0.0  0.0  0.000001
57
2017-12-02 23:09:18.507000
1514.17982239
sum member of class29551
random medoid12854
              0         

1901.88333485
sum member of class31178
random medoid11466
              0         1             2    3    4         5
0  1.402771e-06  0.000901  2.550493e-07  0.0  0.0  0.000001
1  5.100987e-07  0.000967  2.295444e-07  0.0  0.0  0.000001
          0         1             2    3    4             5
0  0.000001  0.000901  2.550493e-07  0.0  0.0  1.275247e-06
1  0.000001  0.004016  3.315641e-07  0.0  0.0  6.886332e-07
75
2017-12-02 23:12:50.356000
1190.39835553
sum member of class27314
random medoid454
          0         1             2    3    4             5
0  0.000001  0.000901  2.550493e-07  0.0  0.0  1.275247e-06
1  0.000001  0.004016  3.315641e-07  0.0  0.0  6.886332e-07
          0         1             2    3    4         5
0  0.000001  0.000901  2.550493e-07  0.0  0.0  0.000001
1  0.000001  0.005991  2.295444e-07  0.0  0.0  0.000001
76
2017-12-02 23:13:02.144000
1201.99694545
sum member of class22358
random medoid2585
          0         1             2    3    4         5
0  0.

...

1070.85812212
sum member of class21601
random medoid17862
          0         1             2    3    4             5
0  0.000001  0.002064  2.550493e-07  0.0  0.0  8.926727e-07
1  0.000002  0.005179  4.080789e-07  0.0  0.0  1.275247e-06
              0         1             2    3    4             5
0  1.198732e-06  0.002064  2.550493e-07  0.0  0.0  8.926727e-07
1  8.416628e-07  0.003208  2.295444e-07  0.0  0.0  1.122217e-06
679
2017-12-03 01:04:20.534000
1329.44017817
sum member of class6111
random medoid4204
              0         1             2    3    4             5
0  1.198732e-06  0.002064  2.550493e-07  0.0  0.0  8.926727e-07
1  8.416628e-07  0.003208  2.295444e-07  0.0  0.0  1.122217e-06
              0         1             2    3    4         5
0  8.416628e-07  0.003962  2.550493e-07  0.0  0.0  0.000002
1  8.416628e-07  0.003208  2.295444e-07  0.0  0.0  0.000001
680
2017-12-03 01:04:30.516000
1219.12329702
sum member of class10787
random medoid7391
              0        

1293.79229696
sum member of class19873
random medoid18977
              0         1             2         3    4         5
0  1.198732e-06  0.006761  2.295444e-07  0.000112  0.0  0.000001
1  8.671677e-07  0.001086  2.550493e-07  0.000000  0.0  0.000001
              0         1             2    3    4         5
0  9.436825e-07  0.004944  2.295444e-07  0.0  0.0  0.000001
1  8.671677e-07  0.001086  2.550493e-07  0.0  0.0  0.000001
697
2017-12-03 01:07:23.541000
1113.75030638
sum member of class8199
random medoid2127
              0         1             2    3    4         5
0  9.436825e-07  0.004944  2.295444e-07  0.0  0.0  0.000001
1  8.671677e-07  0.001086  2.550493e-07  0.0  0.0  0.000001
              0         1             2    3    4         5
0  9.436825e-07  0.004944  2.295444e-07  0.0  0.0  0.000001
1  8.926727e-07  0.000602  2.295444e-07  0.0  0.0  0.000002
698
2017-12-03 01:07:33.729000
1155.03569293
sum member of class25752
random medoid25664
              0         1      

1096.77374923
sum member of class12316
random medoid7864
          0         1             2         3    4             5
0  0.000001  0.006658  1.530296e-07  0.000055  0.0  1.020197e-06
1  0.000001  0.003517  2.295444e-07  0.000000  0.0  8.926727e-07
              0         1             2       3    4             5
0  9.181776e-07  0.004082  2.295444e-07  0.0001  0.0  1.402771e-06
1  1.071207e-06  0.003517  2.295444e-07  0.0000  0.0  8.926727e-07
716
2017-12-03 01:10:35.527000
1259.95272643
sum member of class20527
random medoid3819
              0         1             2       3    4             5
0  9.181776e-07  0.004082  2.295444e-07  0.0001  0.0  1.402771e-06
1  1.071207e-06  0.003517  2.295444e-07  0.0000  0.0  8.926727e-07
              0         1             2    3    4             5
0  6.121184e-07  0.005126  2.550493e-07  0.0  0.0  1.020197e-06
1  1.071207e-06  0.003517  2.295444e-07  0.0  0.0  8.926727e-07
717
2017-12-03 01:10:45.798000
1077.8580712
sum member of class179

1063.23259182
sum member of class12755
random medoid8544
              0         1             2    3    4         5
0  5.611085e-07  0.003120  2.295444e-07  0.0  0.0  0.000001
1  1.071207e-06  0.004853  3.060592e-07  0.0  0.0  0.000001
              0         1             2         3    4             5
0  8.416628e-07  0.003405  2.295444e-07  0.000067  0.0  4.080789e-07
1  1.071207e-06  0.004853  3.060592e-07  0.000000  0.0  1.020197e-06
734
2017-12-03 01:13:37.274000
1102.12771179
sum member of class13327
random medoid3595
              0         1             2         3    4             5
0  8.416628e-07  0.003405  2.295444e-07  0.000067  0.0  4.080789e-07
1  1.071207e-06  0.004853  3.060592e-07  0.000000  0.0  1.020197e-06
              0         1             2    3    4             5
0  6.886332e-07  0.003916  2.805543e-07  0.0  0.0  9.181776e-07
1  1.071207e-06  0.004853  3.060592e-07  0.0  0.0  1.020197e-06
735
2017-12-03 01:13:47.325000
1137.42666952
sum member of class17537

1458.40038547
sum member of class18915
random medoid1668
              0         1             2    3    4         5
0  8.926727e-07  0.000522  2.550493e-07  0.0  0.0  0.000001
1  9.436825e-07  0.007789  3.315641e-07  0.0  0.0  0.000001
              0         1             2    3    4         5
0  8.926727e-07  0.000522  2.550493e-07  0.0  0.0  0.000001
1  1.326257e-06  0.002407  2.295444e-07  0.0  0.0  0.000001
752
2017-12-03 01:16:39.046000
1520.46745599
sum member of class2816
random medoid980
              0         1             2    3    4         5
0  8.926727e-07  0.000522  2.550493e-07  0.0  0.0  0.000001
1  1.326257e-06  0.002407  2.295444e-07  0.0  0.0  0.000001
          0         1             2    3    4         5
0  0.000001  0.003142  2.295444e-07  0.0  0.0  0.000001
1  0.000001  0.002407  2.295444e-07  0.0  0.0  0.000001
753
2017-12-03 01:16:50.447000
1355.20138322
sum member of class25745
random medoid968
          0         1             2    3    4         5
0  0.0

<h1> Train Data - Prepare Prediction Result </h1>

In [9]:
predictedTrainLabel, predictedTrainLabelLevel = DataFrame(trainResult["prediction"])[0].factorize()

<h1> Train Data - Print Prediction Result </h1>

In [10]:
print(metrics.confusion_matrix(trainLabel,predictedTrainLabel))
print(metrics.accuracy_score(trainLabel, predictedTrainLabel))

[[ 9450 15270]
 [ 3067  4774]]
0.436841620343


<h1> Test Data - Load </h1>

In [11]:
testData = read_csv("CencusIncome.csv", header=None)
testLabel = testData.loc[:,6]
testLabel, testLabelLevel = testLabel.factorize()
testData = testData.drop(6, axis = 1)

<h1> Test Data - Preprocess </h1>

In [12]:
normalizedTest = normalize(testData) 

<h1> Test Data - Running </h1>

In [17]:
testResult = k_medoids(normalizedTest, 2, 1)

0
2017-12-03 02:27:52.192000


KeyboardInterrupt: 

<h1> Test Data - Prepare Prediction Result </h1>

In [ ]:
predictionTestLabel, predictionTestLabelLevel = DataFrame(testResult["prediction"])[0].factorize()

<h1> Test Data - Print Prediction Result </h1>

In [ ]:
print(metrics.confusion_matrix(testLabel,predictionTestLabel))
print(metrics.accuracy_score(testLabel, predictionTestLabel))

In [19]:
trainResult2 = k_medoids(trainData, 2, 100)

0
2017-12-03 03:29:33.442000
7448070.63038
    0       1   2  3  4   5
0  31   59083   9  0  0  40
1  21  249271  10  0  0  40
sum member of class12474
random medoid8576
    0       1   2  3  4   5
0  31   59083   9  0  0  40
1  21  249271  10  0  0  40
    0       1   2  3  4   5
0  44  101950  15  0  0  40
1  21  249271  10  0  0  40
1
2017-12-03 03:29:45.847000
6876932.89265
sum member of class16853
random medoid8727
    0       1   2  3  4   5
0  44  101950  15  0  0  40
1  21  249271  10  0  0  40
    0       1   2  3  4   5
0  44  101950  15  0  0  40
1  23  214227  11  0  0  30
2
2017-12-03 03:29:58.370000
6587404.12329
sum member of class19590
random medoid14610
    0       1   2  3  4   5
0  44  101950  15  0  0  40
1  23  214227  11  0  0  30
    0       1   2  3  4   5
0  44  101950  15  0  0  40
1  24  295763  13  0  0  50
3
2017-12-03 03:30:11.278000
7274810.28807
sum member of class12591
random medoid4348
    0       1   2  3  4   5
0  44  101950  15  0  0  40
1  24  2957

6985043.33136
sum member of class6804
random medoid364
    0       1   2  3  4   5
0  55  178644   5  0  0  55
1  31  332379  10  0  0  50
    0       1  2  3  4   5
0  55  178644  5  0  0  55
1  22  410439  9  0  0  55
33
2017-12-03 03:36:41.415000
7334439.99896
sum member of class4641
random medoid887
    0       1  2  3  4   5
0  55  178644  5  0  0  55
1  22  410439  9  0  0  55
    0       1   2  3  4   5
0  55  178644   5  0  0  55
1  33  155781  10  0  0  60
34
2017-12-03 03:36:54.366000
7466459.15511
sum member of class18300
random medoid3650
    0       1   2  3  4   5
0  55  178644   5  0  0  55
1  33  155781  10  0  0  60
    0       1   2  3  4   5
0  18  104704   9  0  0  40
1  33  155781  10  0  0  60
35
2017-12-03 03:37:07.445000
7371770.18491
sum member of class9699
random medoid1875
    0       1   2  3  4   5
0  18  104704   9  0  0  40
1  33  155781  10  0  0  60
    0       1   2  3  4   5
0  63  174018  10  0  0  40
1  33  155781  10  0  0  60
36
2017-12-03 03:37:2

8714916.46338
sum member of class28696
random medoid3204
    0       1   2  3     4   5
0  21  111467  10  0     0  40
1  24   43323   9  0  1762  40
    0       1   2  3     4   5
0  50  100029  13  0     0  65
1  24   43323   9  0  1762  40
66
2017-12-03 03:43:48.951000
9173541.97919
sum member of class29014
random medoid17441
    0       1   2  3     4   5
0  50  100029  13  0     0  65
1  24   43323   9  0  1762  40
    0       1  2  3     4   5
0  32  183801  9  0     0  40
1  24   43323  9  0  1762  40
67
2017-12-03 03:44:04.287000
7040378.00741
sum member of class25198
random medoid23411
    0       1  2  3     4   5
0  32  183801  9  0     0  40
1  24   43323  9  0  1762  40
    0       1   2  3     4   5
0  31  207505  16  0  1977  70
1  24   43323   9  0  1762  40
68
2017-12-03 03:44:18.894000
7227511.60144
sum member of class23386
random medoid2214
    0       1   2  3     4   5
0  31  207505  16  0  1977  70
1  24   43323   9  0  1762  40
    0       1  2     3     4   5
0 

7800961.47245
sum member of class27664
random medoid15489
    0       1   2  3     4   5
0  21  146499  11  0  1579  40
1  58   35723   9  0     0  60
    0       1   2  3     4   5
0  45  233511  13  0  1977  60
1  58   35723   9  0     0  60
98
2017-12-03 03:50:59.152000
7606142.56242
sum member of class10234
random medoid7779
    0       1   2  3     4   5
0  45  233511  13  0  1977  60
1  58   35723   9  0     0  60
    0       1   2  3     4   5
0  45  233511  13  0  1977  60
1  17  287160   7  0     0  15
99
2017-12-03 03:51:12.007000
8750093.18522
sum member of class6540
random medoid1169
    0       1   2  3     4   5
0  45  233511  13  0  1977  60
1  17  287160   7  0     0  15
    0       1   2  3     4   5
0  45  233511  13  0  1977  60
1  40  182302   9  0     0  40
100
2017-12-03 03:51:24.992000
7271650.58652
sum member of class21489
random medoid1440
    0       1   2  3     4   5
0  45  233511  13  0  1977  60
1  40  182302   9  0     0  40
    0       1   2  3     4   5

8070546.42326
sum member of class12382
random medoid2852
    0       1   2  3  4   5
0  69   37745  11  0  0   8
1  39  269186   9  0  0  40
    0       1  2  3  4   5
0  54   32778  9  0  0  30
1  39  269186  9  0  0  40
130
2017-12-03 03:57:51.818000
8159718.94551
sum member of class12083
random medoid2927
    0       1  2  3  4   5
0  54   32778  9  0  0  30
1  39  269186  9  0  0  40
    0       1  2  3  4   5
0  52   91048  9  0  0  35
1  39  269186  9  0  0  40
131
2017-12-03 03:58:05.143000
7242511.0383
sum member of class16033
random medoid2223
    0       1  2  3  4   5
0  52   91048  9  0  0  35
1  39  269186  9  0  0  40
    0       1  2  3  4   5
0  52   91048  9  0  0  35
1  23  336360  4  0  0  40
132
2017-12-03 03:58:17.839000
7856249.49544
sum member of class10437
random medoid4085
    0       1  2  3  4   5
0  52   91048  9  0  0  35
1  23  336360  4  0  0  40
    0       1  2  3  4   5
0  52   91048  9  0  0  35
1  37  243425  9  0  0  50
133
2017-12-03 03:58:30.39700

7137979.42998
sum member of class24095
random medoid13680
    0       1   2  3  4   5
0  67   74335   9  0  0  10
1  38  165930  10  0  0  50
    0      1   2  3  4   5
0  67  74335   9  0  0  10
1  57  31532  13  0  0  40
162
2017-12-03 04:04:41.354000
10290623.3065
sum member of class2491
random medoid2337
    0      1   2  3  4   5
0  67  74335   9  0  0  10
1  57  31532  13  0  0  40
    0       1  2  3  4   5
0  67   74335  9  0  0  10
1  22  230574  6  0  0  25
163
2017-12-03 04:04:53.823000
7020266.78623
sum member of class12253
random medoid5710
    0       1  2  3  4   5
0  67   74335  9  0  0  10
1  22  230574  6  0  0  25
    0       1   2  3  4   5
0  41   27444  10  0  0  46
1  22  230574   6  0  0  25
164
2017-12-03 04:05:06.449000
7613062.79577
sum member of class9563
random medoid9300
    0       1   2  3  4   5
0  41   27444  10  0  0  46
1  22  230574   6  0  0  25
    0       1  2  3  4   5
0  42  126098  9  0  0  40
1  22  230574  6  0  0  25
165
2017-12-03 04:05:19

6595504.79423
sum member of class11314
random medoid4367
    0       1   2  3  4   5
0  46  190115  12  0  0  40
1  53   99476   9  0  0  38
    0       1   2  3  4   5
0  46  190115  12  0  0  40
1  33  256211   9  0  0  40
194
2017-12-03 04:11:29.345000
7244556.9788
sum member of class23135
random medoid18422
    0       1   2  3  4   5
0  46  190115  12  0  0  40
1  33  256211   9  0  0  40
    0       1  2  3  4   5
0  39  252327  5  0  0  35
1  33  256211  9  0  0  40
195
2017-12-03 04:11:42.323000
9766614.623
sum member of class25665
random medoid14217
    0       1  2  3  4   5
0  39  252327  5  0  0  35
1  33  256211  9  0  0  40
    0       1   2  3  4   5
0  53  199665  13  0  0  30
1  33  256211   9  0  0  40
196
2017-12-03 04:11:55.241000
7530286.61333
sum member of class23621
random medoid16758
    0       1   2  3  4   5
0  53  199665  13  0  0  30
1  33  256211   9  0  0  40
    0       1  2     3  4   5
0  38  383239  6  3103  0  40
1  33  256211  9     0  0  40
197
201

7686991.91266
sum member of class12675
random medoid706
    0       1   2  3  4   5
0  30  188398   9  0  0  30
1  43  208277  14  0  0  45
    0       1  2  3  4   5
0  30  188398  9  0  0  30
1  40  229148  8  0  0  40
227
2017-12-03 04:18:32.874000
7402088.14262
sum member of class11074
random medoid8974
    0       1  2  3  4   5
0  30  188398  9  0  0  30
1  40  229148  8  0  0  40
    0       1  2  3  4   5
0  30  188398  9  0  0  30
1  30  226943  9  0  0  35
228
2017-12-03 04:18:44.229000
7424783.6213
sum member of class21352
random medoid9724
    0       1  2  3  4   5
0  30  188398  9  0  0  30
1  30  226943  9  0  0  35
    0       1  2  3  4   5
0  26  176520  9  0  0  53
1  30  226943  9  0  0  35
229
2017-12-03 04:18:55.571000
7136700.01468
sum member of class20455
random medoid9773
    0       1  2  3  4   5
0  26  176520  9  0  0  53
1  30  226943  9  0  0  35
    0       1   2  3  4   5
0  21  149809  12  0  0  40
1  30  226943   9  0  0  35
230
2017-12-03 04:19:07.215

In [ ]:
trainResult3 = k_medoids(normalizedTrain, 2, 10

In [21]:
predictedTrainLabel2, predictedTrainLabelLevel2 = DataFrame(trainResult2["prediction"])[0].factorize()
# predictedTrainLabel3, predictedTrainLabelLevel3 = DataFrame(trainResult3["prediction"])[0].factorize()

In [22]:
print(metrics.confusion_matrix(trainLabel,predictedTrainLabel2))
print(metrics.accuracy_score(trainLabel, predictedTrainLabel2))

[[ 9990 14730]
 [ 3274  4567]]
0.447068578975


In [ ]:
print(metrics.confusion_matrix(trainLabel,predictedTrainLabel3))
print(metrics.accuracy_score(trainLabel, predictedTrainLabel3))